## Anatomy of a NIfTI File

teach you using Python and Jupyter notebooks. We'll start with a basic overview of how we can load, view, and play around with MR images using Python's neuroimaging libraries (<code>Nibabel</code>). 

### Intro to NIfTI

NIfTI is one of the most ubiquitous file formats for storing neuroimaging data. We'll cover a few details to get started working with them. If you're interested in learning more about NIfTI images, we highly recommend [this blog post about the NIfTI format](http://brainder.org/2012/09/23/the-nifti-file-format/).

### Reading NIfTI Images

[NiBabel](http://nipy.org/nibabel/) is a Python package for reading and writing neuroimaging data. To learn more about how NiBabel handles NIfTIs, check out the [Working with NIfTI images](http://nipy.org/nibabel/nifti_images.html) page of the NiBabel documentation.

In [1]:
import nibabel as nib

First, use the `load()` function to create a NiBabel image object from a NIfTI file. We'll load in a T1w image from the dataset we'll be using for this tutorial.

In [3]:
t1_img = nib.load('../data/ds000030/sub-10171/anat/sub-10171_T1w.nii.gz')

FileNotFoundError: No such file or no access: '../data/ds000030/sub-10171/anat/sub-10171_T1w.nii.gz'

The NiBabel image object contains several components called **attributes** in Python's terminology. To see all of these attributes, type `t1_img.` and <kbd>Tab</kbd>.  
There are three main attributes that we'll discuss today:

#### 1. [Header](http://nipy.org/nibabel/nibabel_images.html#the-image-header): contains metadata about the image, such as image dimensions, data type, etc.

In [ ]:
t1_hdr = t1_img.header
print(t1_hdr)

The NiBabel image header is a Python **dictionary**. Dictionaries are containers that hold pairs of objects - keys and values. Let's take a look at all of the keys.

In [ ]:
t1_hdr.keys()

We can access the value stored by a given key by typing:

```python
t1_hdr['<key_name>']
```

<b>EXERCISE:</b> Extract the value of <code>pixdim</code> from <code>t1_hdr</code>

In [ ]:
t1_hdr['pixdim']

#### 2. Data
The data is a multidimensional array representing the image data.

In [ ]:
t1_data = t1_img.get_fdata()
t1_data

In [ ]:
type(t1_data)

Each element in the array corresponds to an intensity value. The range of values typically goes from 0 (black) to 255 (white).

We can check some basic properties of the array.  
The `type()` function will only tell you that a variable is a NumPy array. It won’t tell you the type of data inside of the array.

In [ ]:
t1_data.dtype

This tells us that the NumPy array’s elements are floating-point numbers.   
The data type of an image controls the range of possible intensities. As the number of possible values increases, the amount of space the image takes up in memory also increases.

<b>EXERCISE:</b> How can we see the number of dimensions in the <code>t1_data</code> array? What about the array's shape? Once again, all of the attributes of the array can be seen by typing <code>t1_data.</code> and <kbd>Tab</kbd>.

In [ ]:
t1_data.ndim

In [ ]:
t1_data.shape

The shape of the data always has at least 3 dimensions (X, Y, and Z) and sometimes a 4th, T (time).  
This T1w image has 3 dimensions. This means that the brain was scanned in 176 slices with a resolution of 256 x 256 voxels per slice.

<img src="../static/images/mri_slices.jpg" alt="Drawing" align="middle" width="300px"/>

Sourced from: Tariq, Humera & Burney, S.M.Aqil. (2014). Brain MRI literature review for interdisciplinary studies. Journal of biomedical graphics and computing. 4. 41-53. 10.5430/jbgc.v4n4p41. 

#### 3. [Affine](http://nipy.org/nibabel/coordinate_systems.html): tells the position of the image array data in a *reference space*

How do we identify the location of an object in space? Coordinate systems! The most common coordinate system in neuroimaging is called *RAS* (right (X), anterior (Y), superior (Z)). A coordinate of (2, -10, 5) refers to a point in space 2 mm to the right, 10 mm to the back and 5 mm above. The affine array helps us identify a real location for each voxel in our image data.

<img src="../static/images/coordinate_systems.png" alt="Drawing" align="middle" width="300px"/>

Sourced from: https://www.slicer.org/wiki/Coordinate_systems

In [ ]:
t1_affine = t1_img.affine
t1_affine

### Working With Image Data

#### Slicing

n-dimensional images are just stacks of numpy arrays.  Each value in the array is assigned to an x, y or z coordinate.  
<img src="../static/images/numpy_arrays.png" alt="Drawing" align="middle" width="500px"/>

You'll recall our example T1w image is a 3D image with dimensions $176 \times 256 \times 256$.

Slicing does exactly what it seems to imply. Giving our 3D volume, we pull out a 2D slice of our data. Here's an example of slicing from left to right (**sagittal slicing**):

<img src="https://upload.wikimedia.org/wikipedia/commons/5/56/Parasagittal_MRI_of_human_head_in_patient_with_benign_familial_macrocephaly_prior_to_brain_injury_%28ANIMATED%29.gif"/>

This gif is a series of 2D images or **slices** moving from left to right. We'll now look into how we can pull slices from our 3D image

***
Sourced from: https://en.wikipedia.org/wiki/Neuroimaging#/media/File:Parasagittal_MRI_of_human_head_in_patient_with_benign_familial_macrocephaly_prior_to_brain_injury_(ANIMATED).gif

***

Note that we have two ways of working with neuroimaging data. 


1. We have the actual <code>t1_img</code> we loaded using <code>nibabel</code>
2. We have the array we pulled from <code>t1_img</code> in the form of <code>t1_data</code>


For each of these representations of our data we can pull "slices" using different methods.


First we'll work with the array in the form of <code>t1_img</code>. Since this is a <code>numpy</code> array, we can use the same method as we'd use when dealing with 3D arrays in Python (R and MATLAB are very similar here!).

In [1]:
#Pull the 10th "sagittal slice"
sagittal_slice = t1_data[10,:,:]

Notice the following structure: 

1. We can **index** an array using the array followed by square brackets as follows t1_data[x,y,z]

With our data:
- <code>x</code> is the left to right index
- <code>y</code> is the back to front index
- <code>z</code> is the bottom to top index


So here we're selecting the 10th slice going from left to right (there are 176 slices going from left to right!)

The <code>:</code>, indicates that we want to grab *everything*. 

In plain english we want to:

**Move 10 spots from the left, then grab a full 2D picture here**

<b>EXERCISE:</b> Now try selecting the 70th slice from the back (this is called a **coronal slice**).
It helps to think of this as follows:

Move 70 spots from the back, then grab the full 2D picture here. 

In [ ]:
coronal_slice = t1_data[:,70,:]

Finally try grabbing a **axial slice**, specifically the 126th slice from the bottom:

In [ ]:
axial_slice = t1_data[:,:,126]

As mentioned earlier we can also slice using <code>t1_img</code>, which we loaded through <code>nibabel</code>. It's sometimes nicer to work with this since we don't have to deal with arrays directly like we did with <code>t1_data</code>. 

Slicing with <code>t1_img</code> is just as straightforward and involves using <code>t1_img.slicer[x,y,z]</code>, but with a single caveat which we'll demonstrate here using the 10th slice from the left:

In [ ]:
sagittal_slice = t1_img.slicer[10:11,:,:]

Notice the difference? Here we have to format a sagittal slice as follows:

<code>t1_img.slicer[ (slice) :  (slice# +1) ,:,:]</code>

This is just a result of how <code>slicer</code> is programmed, but everything else remains the same. 

**Exercise**

Use <code>t1_img.slicer</code> to select the:

1. 70th coronal slice from the back
2. 126th axial slice from the bottom

In [2]:
#Coronal slice
coronal_slice = t1_img.slicer[:,70:71,:]

In [3]:
#Axial Slice
axial_slice = t1_img.slicer[:,:,126:127]

We've been slicing and dicing brain images but we have no idea what they look like! In the next section we'll show you how you can visualize brain slices using <code>nibabel</code>!

#### Visualizing
If we wanted to get a quick view of our data, Nibabel provides this functionality through a method called <code>orthoview()</code>. Here's how it looks in action

In [ ]:
import matplotlib.pyplot as plt

### Writing NIfTI Images

Let's save the mask we just created to a file.

In [ ]:
img_mask = nib.Nifti1Image(mask_apply, t1_affine, t1_hdr)
img_mask.to_filename('../data/test_mask.nii.gz')